In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")
name='LoDate'
prefix='SC_'
path = f'/usr/local/airflow/plugins/{prefix}{name}/'

# datenow_min1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 4, 14, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '0 1 * * *',
          tags=[prefix+name, 'Sync', 'at0']
)

In [4]:
sql = \
"""
SELECT
d.BranchID as BranchID,
d.BatNbr,
p.PONbr,
p.Descr,
d.InvtID,
SiteID=d.SiteID,
s.Name,
d.TranDate,
d.Crtd_DateTime,
--d.JrnlType,
--d.TranType,
--T.TranType as ltTranType,
LotSerNbr=isnull(t.LotSerNbr,'')
,ExpDate=isnull(t.ExpDate,''),
Qty = d.CnvFact * d.InvtMult * isnull(T.Qty,d.Qty)
--NhapKhac=0,
--QuyCach=dbo.fr_OM_GetCnvFact(d.InvtID)
FROM dbo.IN_Trans d WITH(NOLOCK)                   
INNER JOIN dbo.IN_LotTrans T WITH(NOLOCK) ON T.BranchID = D.BranchID AND t.BatNbr = d.BatNbr  AND t.RefNbr = d.RefNbr AND t.INTranLineRef = d.LineRef AND t.InvtID = d.InvtID
INNER JOIN dbo.IN_Site s ON d.SiteID = s.SiteId
INNER JOIN PO_Receipt p on p.BranchID = d.BranchID and p.BatNbr = d.BatNbr
--WHERE CAST(d.TranDate AS DATE) BETWEEN CAST('20210501' AS DATE) AND CAST('20220627' AS DATE)
AND d.JrnlType  IN ('PO')
AND d.TranType IN ('RC') AND t.Qty >0          
AND d.Rlsed  =  1       
AND d.InvtMult>0
"""

In [5]:
df = get_ms_df(sql)
bq_values_insert(df, "d_sc_lodate", 3)

1it [00:05,  5.36s/it]


In [7]:
df.head()

,branchid,batnbr,ponbr,descr,invtid,siteid,name,trandate,crtd_datetime,lotsernbr,expdate,qty
0,MR0001,IN21807226,MR0001210727001,,OH051,WH0001,Kho bán hàng Dược Pha Nam,2021-08-09,2021-08-09 10:02:42.720,0150721,2023-07-25,18000.0
1,MR0001,IN21807985,MR0001210727002,,OH023,WH0001,Kho bán hàng Dược Pha Nam,2021-08-10,2021-08-10 15:51:30.403,0030621,2023-06-06,2100.0
2,MR0001,IN21807985,MR0001210727002,,EH086,WH0001,Kho bán hàng Dược Pha Nam,2021-08-10,2021-08-10 15:51:30.417,0070721,2023-07-12,5000.0
3,MR0001,IN21807985,MR0001210727002,,EH087,WH0001,Kho bán hàng Dược Pha Nam,2021-08-10,2021-08-10 15:51:30.433,0040621,2024-06-21,400.0
4,MR0001,IN21807985,MR0001210727002,,EH087,WH0001,Kho bán hàng Dược Pha Nam,2021-08-10,2021-08-10 15:51:30.433,0050621,2024-06-22,2600.0


In [ ]:
def insert():
    print(sql)

def update():
    df = get_ms_df(sql)
    bq_values_insert(df, "d_sc_lodate", 3)


dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

update = PythonOperator(task_id="update", python_callable=update, dag=dag)

dummy_start >> insert >> update